_Importing neccesary libraries_

In [8]:
import pandas as pd 
import numpy as np
from sklearn import model_selection
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from sklearn.metrics import mean_squared_error 


_Coping Data_

In [9]:
data_matches=pd.read_csv('boxing_matches.csv')
c_data=data_matches.copy()


# Exploratory data analysis 

In [10]:

data_matches.info()

pd.set_option('display.max_columns',500)
c_data.describe()
c_data.describe(include='O')
c_data.drop_duplicates(keep='first',inplace=True)
c_data.isnull().sum()

#c_data['kos_B'].value_counts()
#c_data['kos_A'].value_counts()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 387427 entries, 0 to 387426
Data columns (total 26 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   age_A     352888 non-null  float64
 1   age_B     257935 non-null  float64
 2   height_A  249246 non-null  float64
 3   height_B  134640 non-null  float64
 4   reach_A   112342 non-null  float64
 5   reach_B   37873 non-null   float64
 6   stance_A  231009 non-null  object 
 7   stance_B  231009 non-null  object 
 8   weight_A  135573 non-null  float64
 9   weight_B  130358 non-null  float64
 10  won_A     387427 non-null  int64  
 11  won_B     387427 non-null  int64  
 12  lost_A    387427 non-null  int64  
 13  lost_B    387427 non-null  int64  
 14  drawn_A   387427 non-null  int64  
 15  drawn_B   387427 non-null  int64  
 16  kos_A     387427 non-null  int64  
 17  kos_B     387348 non-null  float64
 18  result    387427 non-null  object 
 19  decision  387427 non-null  object 
 20  judg

age_A        31067
age_B       117927
height_A    126527
height_B    232588
reach_A     253598
reach_B     322025
stance_A    144784
stance_B    144784
weight_A    230961
weight_B    235737
won_A            0
won_B            0
lost_A           0
lost_B           0
drawn_A          0
drawn_B          0
kos_A            0
kos_B           77
result           0
decision         0
judge1_A    307696
judge1_B    307870
judge2_A    290913
judge2_B    291220
judge3_A    299462
judge3_B    299646
dtype: int64

# Data Preprocessing

In [11]:
#Removing unnecessary independent variables
c_data.drop(['decision','judge1_A','judge1_B','judge2_A','judge2_B','judge3_A','judge3_B'],axis=1,inplace=True)

c_imputed=c_data.apply(lambda x:x.fillna(x.median()) if x.dtype=='float' else x.fillna(x.value_counts().index[0]))

c_imputed['kos_B'].astype(int)
c_imputed.isnull().sum()
c_imputed.info()

# c_imputed['stance_A'].astype('category')
# c_imputed['stance_B'].astype('category')


<class 'pandas.core.frame.DataFrame'>
Index: 356740 entries, 0 to 387426
Data columns (total 19 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   age_A     356740 non-null  float64
 1   age_B     356740 non-null  float64
 2   height_A  356740 non-null  float64
 3   height_B  356740 non-null  float64
 4   reach_A   356740 non-null  float64
 5   reach_B   356740 non-null  float64
 6   stance_A  356740 non-null  object 
 7   stance_B  356740 non-null  object 
 8   weight_A  356740 non-null  float64
 9   weight_B  356740 non-null  float64
 10  won_A     356740 non-null  int64  
 11  won_B     356740 non-null  int64  
 12  lost_A    356740 non-null  int64  
 13  lost_B    356740 non-null  int64  
 14  drawn_A   356740 non-null  int64  
 15  drawn_B   356740 non-null  int64  
 16  kos_A     356740 non-null  int64  
 17  kos_B     356740 non-null  float64
 18  result    356740 non-null  object 
dtypes: float64(9), int64(7), object(3)
memory usage: 

In [12]:
c_imputed.isnull().sum()

age_A       0
age_B       0
height_A    0
height_B    0
reach_A     0
reach_B     0
stance_A    0
stance_B    0
weight_A    0
weight_B    0
won_A       0
won_B       0
lost_A      0
lost_B      0
drawn_A     0
drawn_B     0
kos_A       0
kos_B       0
result      0
dtype: int64

In [13]:


c_imputed['stance_A']=c_imputed['stance_A'].map({'orthodox':1,'southpaw':0})
c_imputed['stance_B']=c_imputed['stance_B'].map({'orthodox':1,'southpaw':0})
c_imputed['result']=c_imputed['result'].map({'win_A':1,'win_B':2,'draw':0})
c_imputed_new=pd.get_dummies(c_imputed,drop_first=True)


cols=list(c_imputed_new.columns)
features=list(set(cols)-set(c_imputed_new['result']))
y=c_imputed_new['result']
x=c_imputed_new[features]

x_train,x_,y_train,y_=train_test_split(x,y,test_size=0.4,random_state=0)
x_cv,x_test,y_cv,y_test=train_test_split(x_,y_,test_size=0.5,random_state=0)


In [14]:
print(f"Size of test_dataset is{x_train.shape}")
print(f"Size of test_dataset is{x_cv.shape}")
print(f"Size of test_dataset is{x_test.shape}")
del x_,y_

Size of test_dataset is(214044, 19)
Size of test_dataset is(71348, 19)
Size of test_dataset is(71348, 19)


In [15]:
# print(x_train)
print(c_imputed_new['stance_A'])

0         1
1         1
2         1
3         1
4         1
         ..
387422    1
387423    1
387424    1
387425    1
387426    1
Name: stance_A, Length: 356740, dtype: int64


# Model_Selection

_Model_One : Model_NN_N_

In [16]:
Model_NN_N=Sequential([
    Dense(units=25,activation='relu'),
    Dense(units=15,activation='relu'),
    Dense(units=3,activation='linear')
    ])


Model_NN_N.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss=SparseCategoricalCrossentropy(from_logits=True))

history_Model_NN_N=Model_NN_N.fit(x_train,y_train,epochs=10)


Epoch 1/10
6689/6689 [==============================] - 8s 958us/step - loss: 0.8982
Epoch 2/10
6689/6689 [==============================] - 6s 970us/step - loss: 0.4359
Epoch 3/10
6689/6689 [==============================] - 6s 913us/step - loss: 0.2800
Epoch 4/10
6689/6689 [==============================] - 6s 913us/step - loss: 0.1063
Epoch 5/10
6689/6689 [==============================] - 6s 923us/step - loss: 0.0195
Epoch 6/10
6689/6689 [==============================] - 6s 916us/step - loss: 0.0037
Epoch 7/10
6689/6689 [==============================] - 6s 928us/step - loss: 0.0014
Epoch 8/10
6689/6689 [==============================] - 7s 1ms/step - loss: 8.9288e-04
Epoch 9/10
6689/6689 [==============================] - 7s 982us/step - loss: 4.4664e-04
Epoch 10/10
6689/6689 [==============================] - 7s 1ms/step - loss: 6.7318e-04


_training Dataset_

In [17]:
Logit_train=Model_NN_N.predict(x_train)
f_x=tf.nn.softmax(Logit_train).numpy()

train_res=np.argmax(f_x,axis=1)

6689/6689 [==============================] - 7s 990us/step


_cross-validation dataset_

In [18]:
Logit_cv=Model_NN_N.predict(x_cv)
f_x=tf.nn.softmax(Logit_cv).numpy()

cv_res=np.argmax(f_x,axis=1)

2230/2230 [==============================] - 2s 950us/step


_test dataset_

In [19]:
Logit_test=Model_NN_N.predict(x_test)
f_x=tf.nn.softmax(Logit_test).numpy()

test_res=np.argmax(f_x,axis=1)



2230/2230 [==============================] - 2s 958us/step


Evalution Based on accuracy of model

In [20]:
print(test_res)
print(accuracy_score(test_res,y_test))
miss_classified=(y_test!=test_res).sum()
print(miss_classified)

[1 1 1 ... 1 1 1]
0.9999579525704996
3


_MSE's on training,dev,testing dataset_

In [32]:
J_train_error_Model_1=mean_squared_error(y_train,train_res)
J_cv_error_Model_1=mean_squared_error(y_cv,cv_res)
J_test_error_Model_1=mean_squared_error(y_test,test_res)
print(f"Trainig Error { J_train_error_Model_1}")
print(f"Cross-validation Error {J_cv_error_Model_1}")
print(f"Testing Error {J_test_error_Model_1}")


Trainig Error 1.8687746444656237e-05
Cross-validation Error 1.4015809833492179e-05
Testing Error 4.204742950047654e-05


_Model_Two_

In [22]:
Model_NN=Sequential([
    Dense(units=15,activation='relu'),
    Dense(units=10,activation='relu'),
    Dense(units=3,activation='linear')
    ])

Model_NN.compile(optimizer=tf.keras.optimizers.Adam(0.0001),loss=SparseCategoricalCrossentropy(from_logits=True),)

historyModel_NN=Model_NN.fit(x_train,y_train,epochs=10)

Epoch 1/10


6689/6689 [==============================] - 8s 1ms/step - loss: 2.3206
Epoch 2/10
6689/6689 [==============================] - 11s 2ms/step - loss: 0.6768
Epoch 3/10
6689/6689 [==============================] - 8s 1ms/step - loss: 0.5647
Epoch 4/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.5090
Epoch 5/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.4631
Epoch 6/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.4090
Epoch 7/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.3331
Epoch 8/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.2297
Epoch 9/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.1195
Epoch 10/10
6689/6689 [==============================] - 7s 1ms/step - loss: 0.0435


_training Dataset_

In [23]:
Logit_train_N=Model_NN.predict(x_train)
f_x=tf.nn.softmax(Logit_train_N).numpy()

train_res_N=np.argmax(f_x,axis=1)

6689/6689 [==============================] - 6s 953us/step


_cross-validation dataset_

In [24]:
Logit_cv_N=Model_NN.predict(x_cv)
f_x=tf.nn.softmax(Logit_cv_N).numpy()

cv_res_N=np.argmax(f_x,axis=1)

2230/2230 [==============================] - 2s 951us/step


_test dataset_

In [25]:
Logit_test_N=Model_NN.predict(x_test)
f_x=tf.nn.softmax(Logit_test_N).numpy()

test_res_N=np.argmax(f_x,axis=1)


2230/2230 [==============================] - 2s 873us/step


In [29]:
print(test_res_N)
print(accuracy_score(test_res_N,y_test))
miss_classified=(y_test!=test_res_N).sum()
print(miss_classified)

[1 1 1 ... 1 1 1]
0.9993412569378258
47


_MSE's on training,dev,testing dataset_

In [31]:
J_train_error_Model_2=mean_squared_error(y_train,train_res_N)
J_cv_error_Model_2=mean_squared_error(y_cv,cv_res_N)
J_test_error_Model_2=mean_squared_error(y_test,test_res_N)
print(f"Trainig Error for Model_Two : { J_train_error_Model_2}")
print(f"Cross-validation Error for Model_Two : {J_cv_error_Model_2}")
print(f"Testing Error for Model_Two : {J_test_error_Model_2}")


Trainig Error for Model_Two : 0.0006634149987852965
Cross-validation Error for Model_Two : 0.0007288221113415933
Testing Error for Model_Two : 0.0006587430621741324


# Since J_cv_error_Model_1 is smaller,so Model_One is selected as best for our use case

comparing Models based on cv set


# Tyson_fury vs francis_Ngannou Match prediction

In [28]:

data = {
    'age_A': 33,  # Tyson Fury's age
    'age_B': 35,  # Francis Ngannou's age
    'height_A': 210.82,  # Tyson Fury's height in centimeters (1 foot = 30.48 cm)
    'height_B': 193.04,  # Francis Ngannou's height in centimeters (1 foot = 30.48 cm)
    'reach_A': 85,  # Tyson Fury's reach in inches
    'reach_B': 83,  # Francis Ngannou's reach in inches
    'stance_A': 'Orthodox',  # Tyson Fury's fighting stance
    'stance_B': 'Orthodox',  # Francis Ngannou's fighting stance
    'weight_A': 256,  # Tyson Fury's weight in pounds
    'weight_B': 263,  # Francis Ngannou's weight in pounds
    'won_A': 30,  # Tyson Fury's number of wins
    'won_B': 0,  # Francis Ngannou's number of wins in boxing
    'lost_A': 0,  # Tyson Fury's number of losses
    'lost_B': 0,  # Francis Ngannou's number of losses in boxing
    'drawn_A': 1,  # Tyson Fury's number of draws
    'drawn_B': 0,  # Francis Ngannou's number of draws in boxing
    'kos_A': 21,  # Tyson Fury's number of knockouts
    'kos_B': 0,  # Francis Ngannou's number of knockouts in boxing
}

X=np.array([0,33,35,210.82,193.04,85,83,1,1,256,263,30,0,0,0,1,0,21,0])

X = X.reshape(1, -1)
L2=Model_NN_N.predict(X)
f_x_m=tf.nn.softmax(L2).numpy()
print(f_x_m)
res_m=np.argmax(f_x_m,axis=1)
print(res_m)


1/1 [==============================] - 0s 8ms/step
[[0. 0. 1.]]
[2]
